In [3]:
import pandas as pd
from datetime import date, timedelta
import re
from nltk.tokenize import word_tokenize
from gensim import corpora, models
from gensim.models import KeyedVectors
from gensim.matutils import corpus2dense
from huggingface_hub import hf_hub_download
import gensim
import gensim.downloader as api
# import jieba
from gensim.models import Word2Vec, FastText
import numpy as np

In [43]:
class Config:
    
    embedding_size = 300 # embedding size of word embedding
    maxlen = 100 # maximum length of a sentence
    LSTM_output_size = 64 # dimensionality of the output space.
    loss = 'binary_crossentropy'

label = ["han", "nom"]

config = Config()

In [ ]:

# #Loading the word vectors from Google trained word2Vec model
# # model = KeyedVectors.load_word2vec_format(hf_hub_download(repo_id="Word2vec/wikipedia2vec_zhwiki_20180420_100d", filename="zhwiki_20180420_100d.txt"), binary=False)
# # model = api.load("wikipedia2vec_zhwiki_20180420_100")

# # 橘子
# # Sample Chinese sentences
# raw_sentences = [
#     "我吃一個蘋果",
#     "这是一个简单的句子。",
#     "这是另一个例子。",
#     "我们正在学习自然语言处理。",
# ]

# # Tokenize the sentences using jieba
# corpus = [list(jieba.cut(sentence)) for sentence in raw_sentences]


In [ ]:
# vector_size=150

# Train the model
# model = FastText(sentences=corpus, vector_size=config.embedding_size, window=5, min_count=1, workers=4)
from preprocessor import *
from embedding_model import *
from dictionary import *
embedding_model = EmbeddingModel()

In [10]:
embedding_model.load("embedding.model", verbose=True)
print(embedding_model)

Loading model from embedding.model
Model type: <class 'gensim.models.fasttext.FastText'>
Model size: 12474
Vector size: 300



In [13]:
dictionary = Dictionary()
dictionary.load_from_excel("quocngu_sinonom.xlsx", verbose=True)

Dictionary name: quocngu_sinonom.xlsx
Dictionary size: 53097
Dictionary columns: Index(['QuocNgu', 'SinoNom'], dtype='object')



,QuocNgu,SinoNom
0,a,丫
1,a,也
2,a,亜
3,a,亞
4,a,价
...,...,...
53092,ỳnh,璚
53093,ỳnh,瓊
53094,ỳnh,茕
53095,ỷnh,炯


In [15]:
df = pd.read_excel("data/data_collection.xlsx")
df.rename(columns={"Character": "Label"}, inplace=True)
df

,Index,Source,Text,Label
0,1,https://chunom.org/shelf/corpus/1/,埃 喂 傷 𥙩 饒 穷,Nom
1,2,https://chunom.org/shelf/corpus/1/,𨕭 頭 畧 㩫 簮 掑,Nom
2,3,https://chunom.org/shelf/corpus/1/,𥪝 𠀧 𨑮 𦒹 傳 英 攝 固 𨱽 𫨩 空,Nom
3,4,https://chunom.org/shelf/corpus/1/,欺 𢧚 𡗶 拱 朝 𢚸 奇 𠄩,Nom
4,5,https://chunom.org/shelf/corpus/1/,題 𠸜 𡭉 鵲 縁 𡗶 𦓡 𢧚,Nom
...,...,...,...,...
76772,76773,thivien_Phan-Văn-Ái_Hương-Sơn-thi_raw.html,長 帶 天 香 滿 袖 還,Chinese
76773,76774,thivien_Nguyễn-Sưởng_Nguyệt-trung-quế_raw.html,雲 栽 露 種 已 多 時,Chinese
76774,76775,thivien_Nguyễn-Sưởng_Nguyệt-trung-quế_raw.html,清 影 婆 娑 看 轉 奇,Chinese
76775,76776,thivien_Nguyễn-Sưởng_Nguyệt-trung-quế_raw.html,可 是 姮 娥 偏 愛 惜,Chinese


In [16]:
df.rename(columns={"Text": "sentence"}, inplace=True)

In [35]:
data_sentences = list(df['sentence'].values)

In [36]:
# remove empty string in the list
data_sentences = [x for x in data_sentences if type(x) == str]

In [37]:
# remove ' ' in the item of data_sentences
import re
data_sentences = [re.sub(r'\s+', '', item) for item in data_sentences]
data_sentences

['埃喂傷𥙩饒穷',
 '𨕭頭畧㩫簮掑',
 '𥪝𠀧𨑮𦒹傳英攝固𨱽𫨩空',
 '欺𢧚𡗶拱朝𢚸奇𠄩',
 '題𠸜𡭉鵲縁𡗶𦓡𢧚',
 '𠫾𨔈麻吏﨤仙結願',
 '宋珍麻𥙩菊花',
 '沛𨖅使事交和𨑮冬',
 '主韜不義堆塘',
 '買𠳒教訓丁丁',
 '傳尼女則家庭恨灰',
 '傳尼貞鼠摄制',
 '几忠𠊚佞固𡗶於𥪝',
 '傳翹計㐌属𢚸',
 '𢧚庄辰拱在𢚸媄吒',
 '阮生傳意䀡制',
 '𠫾𫥨習學縁𡗶吹𢧚',
 '西楊傳意此䀡',
 '珠竜餒伴朱𢧚〓𠊛',
 '買台𡽫渃車𣾺',
 '中軍傳意仍𠳒𢖵〓',
 '箋花傳意𨔍𨓡',
 '局棋堆妸𦟐紅打制',
 '皇儲傳意天才',
 '詐形爫𡛔此𠳒端荘',
 '征婦傳意朱詳',
 '觀音傳意欺𫢩',
 '氏牟抌覩冤尼庒他',
 '潘陳傳意䀡戈',
 '翁寧傳意此𥋳',
 '揲安𦊚𡎝麻𡗶𡊰功',
 '九夫傳意䀡戈',
 '别包𣉹特室家如𠊚',
 '蓬萊傳意䀡制',
 '准仙𢝙趣結縁買命',
 '𤊰縁秦晋矯命約𬇚',
 '玉𫅜群待價高',
 '女芇咹槾者桃𠇍埃',
 '謀謨麻𥙩特𠊚𡥵官',
 '春情傳𧘇論盘',
 '𣎀𦣰思想事娘𠬠𠄩',
 '芙蓉傳意𦘧撝',
 '俸〓几晋𠊚秦',
 '芳花傳𧘇珠陳義初',
 '黄秀傳𧘇縁於',
 '﨤娘美女𡗶車𠬠茹',
 '勸孝傳𧘇䀡戈',
 '二十四孝英陀拱通',
 '四灵傳𧘇摄穷',
 '每𠊚每我相逢交歡',
 '𠸦朱堆我北南',
 '人縁定吏車爫𠬠边',
 '𠀧生朱院𨑮願',
 '﨤娘買吘根原事情',
 '𥪝𠀧𨑮𦒹傳仃仃',
 '監𡅳客於𨕭𠁀',
 '咹制朱沛别𠳒𫏾𡢐',
 '停輕𧁷渚𢚁𢀭',
 '古初𠳒𠸠𧗱𡢐猪猪',
 '空埃𧁷典𠀧𠁀𦓡𢗼',
 '箕如丐菓埃扜',
 '梗花埃曰别朱世芇',
 '𡶀箕埃撘𦓡高',
 '滝箕𣷭意埃陶𦓡溇',
 '渃潮埃〓埃洃',
 '干𨖲辰世干耒如空',
 '𥪝棱𣘃意埃槞',
 '𩈘𡗶埃𥙩爫篭金烏',
 '𡄎䀡造化机圖',
 '情算拱丕敖𨒣妥𠁀',
 '大生埃拱刼𠊛𦓡催',
 '㗂𠸦項羽英宜埃當',
 '力能舉鼎拔山',
 '吏䀡西代東征',
 '百戰百勝㐌〓𠃩𡗶',
 '埃咍垓下变移',
 '漢王一勝𨖲〓治位',
 '從容固進𠬠欺',
 '計制各〓麻推事𡗶',
 '於𠁀𠇍几𢚁裁',
 '爭坤爭窖埃埋特𡗉',
 '𠇍𠊚咹於别調',
 '塊渡輕㳥吶料滇滇',

In [39]:
# remove '。' in the item of data_sentences
data_sentences = [re.sub(r'。', '', item) for item in data_sentences]
data_sentences

['埃喂傷𥙩饒穷',
 '𨕭頭畧㩫簮掑',
 '𥪝𠀧𨑮𦒹傳英攝固𨱽𫨩空',
 '欺𢧚𡗶拱朝𢚸奇𠄩',
 '題𠸜𡭉鵲縁𡗶𦓡𢧚',
 '𠫾𨔈麻吏﨤仙結願',
 '宋珍麻𥙩菊花',
 '沛𨖅使事交和𨑮冬',
 '主韜不義堆塘',
 '買𠳒教訓丁丁',
 '傳尼女則家庭恨灰',
 '傳尼貞鼠摄制',
 '几忠𠊚佞固𡗶於𥪝',
 '傳翹計㐌属𢚸',
 '𢧚庄辰拱在𢚸媄吒',
 '阮生傳意䀡制',
 '𠫾𫥨習學縁𡗶吹𢧚',
 '西楊傳意此䀡',
 '珠竜餒伴朱𢧚〓𠊛',
 '買台𡽫渃車𣾺',
 '中軍傳意仍𠳒𢖵〓',
 '箋花傳意𨔍𨓡',
 '局棋堆妸𦟐紅打制',
 '皇儲傳意天才',
 '詐形爫𡛔此𠳒端荘',
 '征婦傳意朱詳',
 '觀音傳意欺𫢩',
 '氏牟抌覩冤尼庒他',
 '潘陳傳意䀡戈',
 '翁寧傳意此𥋳',
 '揲安𦊚𡎝麻𡗶𡊰功',
 '九夫傳意䀡戈',
 '别包𣉹特室家如𠊚',
 '蓬萊傳意䀡制',
 '准仙𢝙趣結縁買命',
 '𤊰縁秦晋矯命約𬇚',
 '玉𫅜群待價高',
 '女芇咹槾者桃𠇍埃',
 '謀謨麻𥙩特𠊚𡥵官',
 '春情傳𧘇論盘',
 '𣎀𦣰思想事娘𠬠𠄩',
 '芙蓉傳意𦘧撝',
 '俸〓几晋𠊚秦',
 '芳花傳𧘇珠陳義初',
 '黄秀傳𧘇縁於',
 '﨤娘美女𡗶車𠬠茹',
 '勸孝傳𧘇䀡戈',
 '二十四孝英陀拱通',
 '四灵傳𧘇摄穷',
 '每𠊚每我相逢交歡',
 '𠸦朱堆我北南',
 '人縁定吏車爫𠬠边',
 '𠀧生朱院𨑮願',
 '﨤娘買吘根原事情',
 '𥪝𠀧𨑮𦒹傳仃仃',
 '監𡅳客於𨕭𠁀',
 '咹制朱沛别𠳒𫏾𡢐',
 '停輕𧁷渚𢚁𢀭',
 '古初𠳒𠸠𧗱𡢐猪猪',
 '空埃𧁷典𠀧𠁀𦓡𢗼',
 '箕如丐菓埃扜',
 '梗花埃曰别朱世芇',
 '𡶀箕埃撘𦓡高',
 '滝箕𣷭意埃陶𦓡溇',
 '渃潮埃〓埃洃',
 '干𨖲辰世干耒如空',
 '𥪝棱𣘃意埃槞',
 '𩈘𡗶埃𥙩爫篭金烏',
 '𡄎䀡造化机圖',
 '情算拱丕敖𨒣妥𠁀',
 '大生埃拱刼𠊛𦓡催',
 '㗂𠸦項羽英宜埃當',
 '力能舉鼎拔山',
 '吏䀡西代東征',
 '百戰百勝㐌〓𠃩𡗶',
 '埃咍垓下变移',
 '漢王一勝𨖲〓治位',
 '從容固進𠬠欺',
 '計制各〓麻推事𡗶',
 '於𠁀𠇍几𢚁裁',
 '爭坤爭窖埃埋特𡗉',
 '𠇍𠊚咹於别調',
 '塊渡輕㳥吶料滇滇',

In [40]:
corpus = [list(sentence) for sentence in data_sentences]
corpus

[['埃', '喂', '傷', '𥙩', '饒', '穷'],
 ['𨕭', '頭', '畧', '㩫', '簮', '掑'],
 ['𥪝', '𠀧', '𨑮', '𦒹', '傳', '英', '攝', '固', '𨱽', '𫨩', '空'],
 ['欺', '𢧚', '𡗶', '拱', '朝', '𢚸', '奇', '𠄩'],
 ['題', '𠸜', '𡭉', '鵲', '縁', '𡗶', '𦓡', '𢧚'],
 ['𠫾', '𨔈', '麻', '吏', '﨤', '仙', '結', '願'],
 ['宋', '珍', '麻', '𥙩', '菊', '花'],
 ['沛', '𨖅', '使', '事', '交', '和', '𨑮', '冬'],
 ['主', '韜', '不', '義', '堆', '塘'],
 ['買', '𠳒', '教', '訓', '丁', '丁'],
 ['傳', '尼', '女', '則', '家', '庭', '恨', '灰'],
 ['傳', '尼', '貞', '鼠', '摄', '制'],
 ['几', '忠', '𠊚', '佞', '固', '𡗶', '於', '𥪝'],
 ['傳', '翹', '計', '㐌', '属', '𢚸'],
 ['𢧚', '庄', '辰', '拱', '在', '𢚸', '媄', '吒'],
 ['阮', '生', '傳', '意', '䀡', '制'],
 ['𠫾', '𫥨', '習', '學', '縁', '𡗶', '吹', '𢧚'],
 ['西', '楊', '傳', '意', '此', '䀡'],
 ['珠', '竜', '餒', '伴', '朱', '𢧚', '〓', '𠊛'],
 ['買', '台', '𡽫', '渃', '車', '𣾺'],
 ['中', '軍', '傳', '意', '仍', '𠳒', '𢖵', '〓'],
 ['箋', '花', '傳', '意', '𨔍', '𨓡'],
 ['局', '棋', '堆', '妸', '𦟐', '紅', '打', '制'],
 ['皇', '儲', '傳', '意', '天', '才'],
 ['詐', '形', '爫', '𡛔', '此', '𠳒', '端', '荘'],
 ['征', '婦', '傳', '意', '朱', '詳'

In [41]:
from keras_preprocessing.text import Tokenizer
from keras_preprocessing import sequence

# Define the OOV token (e.g., "<UNK>")
oov_token = "<UNK>"

tokenizer = Tokenizer(char_level=True, oov_token=oov_token)
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)

In [ ]:
# # Example: Get a vector for a word
# vector = model.wv["简单"]  # Example word: "简单" (simple)
# print("Vector for '简单':", vector.shape)

Vector for '简单': (150,)


In [ ]:
# # Assuming `model.wv` provides word vectors
# vector1 = np.array(model.wv['蘋果'])
# vector2 = np.array(model.wv['橘子'])

# # Normalize the vectors
# vector1 = vector1 / np.linalg.norm(vector1)
# vector2 = vector2 / np.linalg.norm(vector2)

# # Compute the cosine similarity
# cosine_similarity = np.dot(vector1, vector2)
# print(cosine_similarity)

-0.04673171


In [ ]:
# from keras_preprocessing.text import Tokenizer
# from keras_preprocessing import sequence

# # Define the OOV token (e.g., "<UNK>")
# oov_token = "<UNK>"

# tokenizer = Tokenizer(char_level=True, oov_token=oov_token)
# tokenizer.fit_on_texts(corpus)
# sequences = tokenizer.texts_to_sequences(corpus)

# X = sequence.pad_sequences(sequences, maxlen=config.maxlen)
# print('X:\n', X[:10])

X:
 [[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   4  5  6  7]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  8  3
   9 10 11  2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 12 13
  14  3 15  2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  

In [ ]:
# pip install Keras-Preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [45]:
# Get the word index from tokenizer
word_index = tokenizer.word_index
print(word_index)
vocab_size = len(word_index) + 1  # Plus 1 because indices start from 1

# Initialize the embedding matrix with zeros
embedding_matrix = np.zeros((vocab_size, config.embedding_size))

# Fill the embedding matrix
for word, i in word_index.items():
    try:
        # Get the FastText vector for the word
        embedding_vector = embedding_model.model.wv[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        # If no embedding is found for a word, leave the vector as zero
        embedding_matrix[i] = np.zeros(config.embedding_size)

{'<UNK>': 1, '之': 2, '以': 3, '人': 4, '月': 5, '不': 6, '十': 7, '固': 8, '年': 9, '帝': 10, '為': 11, '公': 12, '有': 13, '時': 14, '大': 15, '朱': 16, '如': 17, '二': 18, '吏': 19, '日': 20, '兵': 21, '其': 22, '子': 23, '於': 24, '天': 25, '一': 26, '國': 27, '等': 28, '太': 29, '王': 30, '㐌': 31, '官': 32, '-': 33, '埃': 34, '三': 35, '爲': 36, '軍': 37, '𢚸': 38, '事': 39, '而': 40, '生': 41, '明': 42, '𠊛': 43, '安': 44, '浪': 45, '使': 46, '阮': 47, '文': 48, '行': 49, '曰': 50, '爫': 51, '自': 52, '至': 53, '者': 54, '尼': 55, '拯': 56, '麻': 57, '臣': 58, '及': 59, '民': 60, '皇': 61, '道': 62, '群': 63, '上': 64, '意': 65, '也': 66, '命': 67, '所': 68, '中': 69, '渃': 70, '山': 71, '士': 72, '拱': 73, '是': 74, '特': 75, '共': 76, '買': 77, '空': 78, '黎': 79, '春': 80, '五': 81, '下': 82, '花': 83, '無': 84, '南': 85, '四': 86, '蔑': 87, '外': 88, '名': 89, '情': 90, '分': 91, '元': 92, '世': 93, '朝': 94, '𡗶': 95, '城': 96, '仍': 97, '羅': 98, '些': 99, '平': 100, '書': 101, '宗': 102, '身': 103, '郡': 104, '沛': 105, '武': 106, '各': 107, '別': 108, '得': 109, '都': 110, '𢧚'

In [57]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM, Bidirectional

classifierModel = Sequential()
classifierModel.add(Embedding(vocab_size, config.embedding_size, input_length=config.maxlen, weights=[embedding_matrix], trainable=False))
classifierModel.add(Bidirectional(LSTM(config.LSTM_output_size, return_sequences=True, input_shape=(config.maxlen, ))))
classifierModel.add(Flatten())
classifierModel.add(Dense(1, activation='sigmoid'))

classifierModel.compile(optimizer='adam', loss=config.loss, metrics=['acc'])
classifierModel.summary()


e:\study 7\NLP\Thuc_hanh\nlpenv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
e:\study 7\NLP\Thuc_hanh\nlpenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │     4,514,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,514,100 (17.22 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,514,100 (17.22 MB)

In [49]:
df['BinaryLabel'] = df['Label'].apply(lambda x: 1 if x == 'Nom' else 0)
df

,Index,Source,sentence,Label,BinaryLabel
0,1,https://chunom.org/shelf/corpus/1/,埃 喂 傷 𥙩 饒 穷,Nom,1
1,2,https://chunom.org/shelf/corpus/1/,𨕭 頭 畧 㩫 簮 掑,Nom,1
2,3,https://chunom.org/shelf/corpus/1/,𥪝 𠀧 𨑮 𦒹 傳 英 攝 固 𨱽 𫨩 空,Nom,1
3,4,https://chunom.org/shelf/corpus/1/,欺 𢧚 𡗶 拱 朝 𢚸 奇 𠄩,Nom,1
4,5,https://chunom.org/shelf/corpus/1/,題 𠸜 𡭉 鵲 縁 𡗶 𦓡 𢧚,Nom,1
...,...,...,...,...,...
76772,76773,thivien_Phan-Văn-Ái_Hương-Sơn-thi_raw.html,長 帶 天 香 滿 袖 還,Chinese,0
76773,76774,thivien_Nguyễn-Sưởng_Nguyệt-trung-quế_raw.html,雲 栽 露 種 已 多 時,Chinese,0
76774,76775,thivien_Nguyễn-Sưởng_Nguyệt-trung-quế_raw.html,清 影 婆 娑 看 轉 奇,Chinese,0
76775,76776,thivien_Nguyễn-Sưởng_Nguyệt-trung-quế_raw.html,可 是 姮 娥 偏 愛 惜,Chinese,0


In [51]:
df = df[df['sentence'].apply(lambda x: isinstance(x, str))]

In [52]:
def clean_text(text):
    text = re.sub(r'\s+', '', text)
    text = re.sub(r'。', '', text)
    return text

df['sentence_cleaned'] = df['sentence'].apply(lambda x: clean_text(x))

C:\Users\Admin\AppData\Local\Temp\ipykernel_19780\3449301554.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentence_cleaned'] = df['sentence'].apply(lambda x: clean_text(x))


In [54]:

from sklearn.model_selection import train_test_split

# data = {
#     'Review': [
#         '这个电影非常好看', '我不喜欢这部电影', '剧情很紧张', '演员表现一般', '非常感动', 
#         '无聊的电影', '值得一看', '完全浪费时间', '画面很美', '很糟糕的表演'
#     ],
#     'Liked': [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
# }

# sequences = tokenizer.texts_to_sequences(data['Review'],)
sequences = tokenizer.texts_to_sequences(df['sentence_cleaned'],)

X_data = sequence.pad_sequences(sequences, maxlen=config.maxlen)
print(X_data.shape)
y_data = np.array(df['BinaryLabel'])

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=101)


(76739, 100)


In [58]:
classifierModel.fit(X_train, y_train, epochs=2, batch_size=32)


Epoch 1/2
1679/1679 ━━━━━━━━━━━━━━━━━━━━ 99s 57ms/step - acc: 0.9586 - loss: 0.1090
Epoch 2/2
1679/1679 ━━━━━━━━━━━━━━━━━━━━ 108s 64ms/step - acc: 0.9828 - loss: 0.0444


In [60]:
from sklearn.metrics import confusion_matrix, classification_report

predictions = classifierModel.predict(X_test)
print('predict:', predictions)
predictions = np.round(predictions)
print('predictions:', predictions.flatten())
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

720/720 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step
predict: [[0.9999997 ]
 [0.99749607]
 [0.99999994]
 ...
 [1.        ]
 [1.        ]
 [0.999999  ]]
predictions: [1. 1. 1. ... 1. 1. 1.]
[[ 6642   236]
 [  312 15832]]
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      6878
           1       0.99      0.98      0.98     16144

    accuracy                           0.98     23022
   macro avg       0.97      0.97      0.97     23022
weighted avg       0.98      0.98      0.98     23022



In [64]:
X_test.shape

(23022, 100)

In [63]:
X_train.shape

(53717, 100)

In [65]:
X_test

array([[   0,    0,    0, ..., 1131,   17,  131],
       [   0,    0,    0, ...,  239,  309,   71],
       [   0,    0,    0, ...,  534, 1116, 3199],
       ...,
       [   0,    0,    0, ...,  137, 1914,   19],
       [   0,    0,    0, ...,  194, 1132,  171],
       [   0,    0,    0, ...,  671,  555, 1206]])

In [66]:
predictions

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [67]:
y_test

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [71]:
test_sample = "𡦂才𡦂命窖𱺵恄饒"
test_sample = clean_text(test_sample)
test_sample_sequence = tokenizer.texts_to_sequences([test_sample])

test_sample_data = sequence.pad_sequences(test_sample_sequence, maxlen=config.maxlen)
print("Test sample sequence:", test_sample_sequence)
print("Test sample data:", test_sample_data)

Test sample sequence: [[1072, 211, 1072, 67, 573, 1, 2654, 157]]
Test sample data: [[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0 1072  211 1072   67  573    1
  2654  157]]


In [72]:
pred_test_sample = classifierModel.predict(test_sample_data)
print("Predicted label:", np.round(pred_test_sample).flatten())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Predicted label: [1.]


In [73]:
test_sample = "知章騎馬似乘船"
test_sample = clean_text(test_sample)
test_sample_sequence = tokenizer.texts_to_sequences([test_sample])

test_sample_data = sequence.pad_sequences(test_sample_sequence, maxlen=config.maxlen)
print("Test sample sequence:", test_sample_sequence)
print("Test sample data:", test_sample_data)
pred_test_sample = classifierModel.predict(test_sample_data)
print("Predicted label:", np.round(pred_test_sample).flatten())

Test sample sequence: [[130, 555, 1673, 310, 1141, 789, 266]]
Test sample data: [[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0  130  555 1673  310 1141
   789  266]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Predicted label: [0.]


In [74]:
test_sample = "諭諸裨將檄文"
test_sample = clean_text(test_sample)
test_sample_sequence = tokenizer.texts_to_sequences([test_sample])

test_sample_data = sequence.pad_sequences(test_sample_sequence, maxlen=config.maxlen)
print("Test sample sequence:", test_sample_sequence)
print("Test sample data:", test_sample_data)
pred_test_sample = classifierModel.predict(test_sample_data)
print("Predicted label:", np.round(pred_test_sample).flatten())

Test sample sequence: [[683, 127, 3420, 317, 3623, 48]]
Test sample data: [[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0  683  127 3420  317
  3623   48]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Predicted label: [0.]


In [75]:
test_sample = f"世紀率領陳朝軍隊，成功擊退蒙古軍隊的兩次入侵"
test_sample = clean_text(test_sample)
test_sample_sequence = tokenizer.texts_to_sequences([test_sample])

test_sample_data = sequence.pad_sequences(test_sample_sequence, maxlen=config.maxlen)
print("Test sample sequence:", test_sample_sequence)
print("Test sample data:", test_sample_data)
pred_test_sample = classifierModel.predict(test_sample_data)
print("Predicted label:", np.round(pred_test_sample).flatten())

Test sample sequence: [[93, 812, 451, 285, 112, 94, 37, 312, 336, 298, 114, 626, 749, 1244, 364, 37, 312, 2785, 1887, 467, 146, 911]]
Test sample data: [[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   93  812  451  285  112   94
    37  312  336  298  114  626  749 1244  364   37  312 2785 1887  467
   146  911]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Predicted label: [0.]
